In [2]:
import os
from torch.utils.data import DataLoader
from dataloader_csv import create_dataloader_csv
import matplotlib.pyplot as plt
from PIL import ImageOps
import torchvision
import numpy as np
import cv2

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import *


# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# CPUデバイスを指定
device = torch.device("cpu")

print(device)
path = os.getcwd()
path = '/root/notebooks/sample/dataset/CSV_4label'
train_dataloader,test_dataloader,labels=create_dataloader_csv(path,1,None)
print(train_dataloader)

cpu
Classification label : {'Lateral': 0, 'Posterior': 1, 'Anterior': 2, 'Inferior': 3}
torch.Size([149, 12, 500])
149
test1
dataset len : 149
labels len : 149
tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3])
test2
indices
119
30


In [ ]:
# attention

class Attention_Input(nn.Module):
    def __init__(self):
        super(Attention_Input, self).__init__()
        self.cls_token = nn.Parameter(torch.randn(32))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x と cls_token を連結
        z_0 = torch.cat([self.cls_token.unsqueeze(0), x], dim=0)
        return z_0
# AttentionInputをnn.Moduleに変更
class AttentionInput(nn.Module):
    def __init__(self):
        super(AttentionInput, self).__init__()
        self.attention_input = Attention_Input()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # AttentionInputのforwardメソッドを呼び出す
        output = self.attention_input(x)
        return output

class AttentionLayer(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(AttentionLayer, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads

        # MultiheadAttentionのためにクエリ、キー、バリューの変換を行う層を定義
        self.query_transform = nn.Linear(embed_dim, embed_dim)
        self.key_transform = nn.Linear(embed_dim, embed_dim)
        self.value_transform = nn.Linear(embed_dim, embed_dim)

        # MultiheadAttentionレイヤー
        self.multihead_attention = nn.MultiheadAttention(embed_dim, num_heads)

    def forward(self, x):
        # クエリ、キー、バリューの変換
        query = self.query_transform(x)
        key = self.key_transform(x)
        value = self.value_transform(x)

        # MultiheadAttentionの適用
        attention_output, _ = self.multihead_attention(query, key, value)
        return attention_output

# # Attentionの出力からCLSトークンを取り出す    
class Take_cls(nn.Module):
    def forward(self, x):
        return x[0, :].reshape(1, 32)